In [5]:
import numpy as np
import sounddevice as sd
import threading
import queue
import time
from obspy.clients.seedlink.easyseedlink import EasySeedLinkClient
from obspy.core import Trace, Stream

In [7]:
signal_queue = queue.Queue()
audio_queue = queue.Queue()

In [8]:
# 📡 SeedLink-Client für Live-Daten
class MySeedLinkClient(EasySeedLinkClient):
    def __init__(self, network, station, channel):
        super().__init__("rtserve.iris.washington.edu:18000")
        self.select_stream(network, station, channel)

    def on_data(self, trace):
        fs_signal = trace.stats.sampling_rate
        data = trace.data.astype(np.float32)

        # Normalisierung (optional)
        if np.max(np.abs(data)) > 0:
            data /= np.max(np.abs(data))

        print(f"📡 {trace.id} | {len(data)} samples @ {fs_signal} Hz")
        signal_queue.put((data, fs_signal))


In [9]:
def start_all():
    # Starte SeedLink-Client
    threading.Thread(target=lambda: MySeedLinkClient("IU", "ANMO", "BHZ").run(), daemon=True).start()


In [12]:
from obspy.clients.seedlink.easyseedlink import EasySeedLinkClient

class ListingClient(EasySeedLinkClient):
    def on_seedlink_info(self, info):
        print("📋 Verfügbare Streams:")
        for line in info.strip().splitlines():
            print(line)

client = ListingClient("rtserve.iris.washington.edu:18000")
client.get_info()


TypeError: EasySeedLinkClient.get_info() missing 1 required positional argument: 'level'